In [2]:
import torch

In [4]:
a = torch.tensor([1, 2, 3], dtype=torch.float32)
b = torch.tensor([4, 5, 6], dtype=torch.float32)
a, b

(tensor([1., 2., 3.]), tensor([4., 5., 6.]))

In [20]:
aa = torch.nn.functional.normalize(a.unsqueeze(0))
ba = torch.nn.functional.normalize(b.unsqueeze(0))
aa, ba

(tensor([[0.2673, 0.5345, 0.8018]]), tensor([[0.4558, 0.5698, 0.6838]]))

In [21]:
aa - ba

tensor([[-0.1886, -0.0353,  0.1180]])

In [22]:
aa.sub(ba)

tensor([[-0.1886, -0.0353,  0.1180]])

In [23]:
aa.sub(ba).norm(2)

tensor(0.2252)

In [24]:
aa.sub(ba).norm(2).div(2)

tensor(0.1126)

In [25]:
aa.sub(ba).norm(2).div(2).arcsin()

tensor(0.1129)

In [26]:
x = torch.tensor(2., requires_grad=True)

a = torch.add(x, 1)
b = torch.add(x, 2)
y = torch.mul(a, b)

y.backward()
print(x.grad)

tensor(7.)


In [27]:
print("requires_grad: ", x.requires_grad, a.requires_grad, b.requires_grad, y.requires_grad)
print("is_leaf: ", x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)
print("grad: ", x.grad, a.grad, b.grad, y.grad)


requires_grad:  True True True True
is_leaf:  True False False False
grad:  tensor(7.) None None None


/tmp/ipykernel_12627/3011949837.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print("grad: ", x.grad, a.grad, b.grad, y.grad)


In [30]:
x = torch.tensor(2., requires_grad=True)

a = torch.add(x, 1)
b = torch.add(x, 2)
y = torch.mul(a, b)

grad = torch.autograd.grad(outputs=y, inputs=x)
print(grad[0])
grad

tensor(7.)


(tensor(7.),)

In [32]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x, retain_graph=True)
grad_y = torch.autograd.grad(outputs=z, inputs=y)
print(grad_x[0], grad_y[0])

tensor(12.) tensor(4.)


In [3]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x, retain_graph=True)
grad_xx = torch.autograd.grad(outputs=grad_x, inputs=x)
print(grad_x[0], grad_xx[0])

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [4]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad_x = torch.autograd.grad(outputs=z, inputs=x, create_graph=True)
grad_xx = torch.autograd.grad(outputs=grad_x, inputs=x)
print(grad_x[0], grad_xx[0])

tensor(12., grad_fn=<AddBackward0>) tensor(6.)


In [9]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

grad = torch.autograd.grad(outputs=z, inputs=x, create_graph=True)
grad[0].backward()
print(x.grad)

tensor(6.)


In [13]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

z.backward(create_graph=True)
grad_xx = torch.autograd.grad(outputs=x.grad, inputs=x)
print(x, grad_x[0], grad_xx[0])

tensor(2., requires_grad=True) tensor(12., grad_fn=<AddBackward0>) tensor(6.)


In [16]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

z.backward(create_graph=True)
x.grad.backward()
print(x, x.grad)

tensor(2., requires_grad=True) tensor(18., grad_fn=<CopyBackwards>)


In [17]:
x = torch.tensor(2.).requires_grad_()
y = torch.tensor(3.).requires_grad_()

z = x * x * y

z.backward(create_graph=True)
x.grad.data.zero_()
x.grad.backward()
print(x, x.grad)

tensor(2., requires_grad=True) tensor(6., grad_fn=<CopyBackwards>)


In [19]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x + 1

y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [20]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x + 1

y.sum().backward()
print(x.grad)

tensor([1., 1., 1.])


In [21]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x * x

y.sum().backward()
print(x.grad)

tensor([2., 4., 6.])


In [24]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y: torch.Tensor  = x * x

y.backward(gradient=torch.ones_like(x)) # for non-scalar output
print(x.grad)

tensor([2., 4., 6.])


In [29]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y: torch.Tensor  = x * x

grad_x = torch.autograd.grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y)) 
print(grad_x)

(tensor([2., 4., 6.]),)


In [30]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y: torch.Tensor  = x * x

grad_x = torch.autograd.grad(outputs=y.sum(), inputs=x) 
print(grad_x)

(tensor([2., 4., 6.]),)


In [31]:
x = torch.tensor([2.], requires_grad=True)

a = torch.add(x, 1).detach()
b = torch.add(x, 2)
y = torch.mul(a, b)

y.backward()

print("requires_grad: ", x.requires_grad, a.requires_grad, b.requires_grad, y.requires_grad)
print("is_leaf: ", x.is_leaf, a.is_leaf, b.is_leaf, y.is_leaf)
print("grad: ", x.grad, a.grad, b.grad, y.grad)


requires_grad:  True False True True
is_leaf:  True True False False
grad:  tensor([3.]) None None None


/tmp/ipykernel_20512/2370103711.py:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print("grad: ", x.grad, a.grad, b.grad, y.grad)


In [13]:
torch.nn.functional.normalize(a.unsqueeze(0), dim=2)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [14]:
a / a.sum()

tensor([0.1667, 0.3333, 0.5000])

In [11]:
(a - a.mean()) / a.std()

tensor([-1.,  0.,  1.])

In [12]:
(a - a.min()) / (a.max() - a.min())

tensor([0.0000, 0.5000, 1.0000])

In [18]:
a ** 2

tensor([1., 4., 9.])

In [19]:
(a ** 2).sum().sqrt()

tensor(3.7417)

In [16]:
a.norm(2)

tensor(3.7417)

In [17]:
a / a.norm(2)

tensor([0.2673, 0.5345, 0.8018])